## Predecir si una masa de mamografía es benigna o maligna

*Utilizaremos el conjunto de datos públicos de "masas mamográficas" del repositorio UCI (fuente: https://archive.ics.uci.edu/ml/datasets/Mammographic+Mass)*

*Estos datos contienen 961 instancias de masas detectadas en mamografías y contienen los siguientes atributos:*

   1. Evaluación BI-RADS: 1 a 5 (ordinal)  
   2. Edad: edad del paciente en años (entero)
   3. Forma: forma de masa: redonda = 1 oval = 2 lobular = 3 irregular = 4 (nominal)
   4. Margen: margen de masa: circunscrito = 1 microlobulado = 2 oscurecido = 3 mal definido = 4 espiculado = 5 (nominal)
   5. Densidad: densidad de masa alta = 1 iso = 2 baja = 3 que contiene grasa = 4 (ordinal)
   6. Gravedad: benigna = 0 o maligna = 1 (binominal)

*BI-RADS es una evaluación de cuán segura es la clasificación de gravedad; No es un atributo "predictivo" y, por lo tanto, lo descartaremos. Los atributos de edad, forma, margen y densidad son las características con las que construiremos nuestro modelo, y "gravedad" es la clasificación que intentaremos predecir en función de esos atributos.*


*Aunque "forma" y "margen" son tipos de datos nominales, que sklearn generalmente no trata bien, están lo suficientemente cerca del ordinal como para que no deberíamos descartarlos. La "forma", por ejemplo, se ordena cada vez más de redonda a irregular.*

*Una gran cantidad de angustia y cirugía innecesarias surgen de falsos positivos que surgen de los resultados de la mamografía. Si podemos construir una mejor manera de interpretarlos a través del aprendizaje automático supervisado, podría mejorar muchas vidas.*


## Tu tarea

*Aplique varias técnicas diferentes de aprendizaje automático supervisado a este conjunto de datos y vea cuál produce la mayor precisión medida con la validación cruzada K-Fold (K = 10). Aplicar:*

    * Decision tree
    * Random forest
    * KNN
    * Naive Bayes
    * SVM
    * Logistic Regression
    * And, as a bonus challenge, a neural network using Keras.
    
*Los datos deben limpiarse; Muchas filas contienen datos faltantes, y también puede haber datos erróneos identificables como valores atípicos.*

*Recuerde que algunas técnicas como SVM también requieren que los datos de entrada se normalicen primero.*

*Muchas técnicas también tienen "hiperparámetros" que necesitan ser ajustados. Una vez que identifique un enfoque prometedor, vea si puede mejorarlo aún más ajustando sus hiperparámetros.*


*Pude lograr más del 80% de precisión, ¿puedes superar eso?*

*A continuación he configurado un esquema de un cuaderno para este proyecto, con algunas instrucciones y sugerencias. Si estás preparado para un verdadero desafío, ¡intenta hacer este proyecto desde cero en un cuaderno nuevo y limpio!*


## Comencemos: prepara tus datos

*Comience importando el archivo mammographic_masses.data.txt a un marco de datos Pandas (sugerencia: use read_csv) y échele un vistazo.*

In [39]:
import pandas as pd

df = pd.read_csv(r'./mammographic_masses.data.txt', na_values=['?'], names = ['BI-RADS', 'age', 'shape', 'margin', 'density', 'severity'])
df.head()

,BI-RADS,age,shape,margin,density,severity
0,5.0,67.0,3.0,5.0,3.0,1
1,4.0,43.0,1.0,1.0,NaN,1
2,5.0,58.0,4.0,5.0,3.0,1
3,4.0,28.0,1.0,1.0,3.0,0
4,5.0,74.0,1.0,5.0,NaN,1


*Evaluar si los datos necesitan limpieza; Su modelo es tan bueno como los datos que se le dan. Sugerencia: use describe() en el marco de datos.*

In [40]:
df.describe()

,BI-RADS,age,shape,margin,density,severity
count,959.000000,956.000000,930.000000,913.000000,885.000000,961.000000
mean,4.348279,55.487448,2.721505,2.796276,2.910734,0.463059
std,1.783031,14.480131,1.242792,1.566546,0.380444,0.498893
min,0.000000,18.000000,1.000000,1.000000,1.000000,0.000000
25%,4.000000,45.000000,2.000000,1.000000,3.000000,0.000000
50%,4.000000,57.000000,3.000000,3.000000,3.000000,0.000000
75%,5.000000,66.000000,4.000000,4.000000,3.000000,1.000000
max,55.000000,96.000000,4.000000,5.000000,4.000000,1.000000


*Faltan bastantes valores en el conjunto de datos. Antes de eliminar cada fila que falta de datos, asegurémonos de no sesgar nuestros datos al hacerlo. ¿Parece haber algún tipo de correlación con qué tipo de datos faltan campos? Si lo hubiera, tendríamos que tratar de volver atrás y completar esos datos.*

In [41]:
df.loc[(df['age'].isnull()) |
       (df['shape'].isnull()) |
       (df['margin'].isnull()) |
       (df['density'].isnull())]

,BI-RADS,age,shape,margin,density,severity
1,4.0,43.0,1.0,1.0,NaN,1
4,5.0,74.0,1.0,5.0,NaN,1
5,4.0,65.0,1.0,NaN,3.0,0
6,4.0,70.0,NaN,NaN,3.0,0
7,5.0,42.0,1.0,NaN,3.0,0
...,...,...,...,...,...,...
778,4.0,60.0,NaN,4.0,3.0,0
819,4.0,35.0,3.0,NaN,2.0,0
824,6.0,40.0,NaN,3.0,4.0,1
884,5.0,NaN,4.0,4.0,3.0,1


*Si los datos que faltan parecen distribuidos aleatoriamente, continúe y elimine las filas con los datos que faltan. Sugerencia: use dropna().*

In [42]:
df.dropna(inplace=True)
df.describe()

,BI-RADS,age,shape,margin,density,severity
count,830.000000,830.000000,830.000000,830.000000,830.000000,830.000000
mean,4.393976,55.781928,2.781928,2.813253,2.915663,0.485542
std,1.888371,14.671782,1.242361,1.567175,0.350936,0.500092
min,0.000000,18.000000,1.000000,1.000000,1.000000,0.000000
25%,4.000000,46.000000,2.000000,1.000000,3.000000,0.000000
50%,4.000000,57.000000,3.000000,3.000000,3.000000,0.000000
75%,5.000000,66.000000,4.000000,4.000000,3.000000,1.000000
max,55.000000,96.000000,4.000000,5.000000,4.000000,1.000000


*A continuación, deberá convertir los marcos de datos de Pandas en matrices numpy que puedan ser utilizadas por scikit_learn. Cree una matriz que extraiga solo los datos de entidades con los que queremos trabajar (antigüedad, forma, margen y densidad) y otra matriz que contenga las clases (gravedad). También necesitará una matriz de las etiquetas de nombre de las características.*

In [43]:
valores = df[['age', 'shape',
                             'margin', 'density']].values


clases = df['severity'].values

nomCaract = ['age', 'shape', 'margin', 'density']

valores

array([[67.,  3.,  5.,  3.],
       [58.,  4.,  5.,  3.],
       [28.,  1.,  1.,  3.],
       ...,
       [64.,  4.,  5.,  3.],
       [66.,  4.,  5.,  3.],
       [62.,  3.,  3.,  3.]])

*Algunos de nuestros modelos requieren que los datos de entrada se normalicen, así que adelante y normalice los datos de atributos. Sugerencia: use el preprocesamiento. StandardScaler().*

In [44]:
from sklearn import preprocessing

scaler = preprocessing.StandardScaler()
valores_scaled = scaler.fit_transform(valores)
valores_scaled

array([[ 0.7650629 ,  0.17563638,  1.39618483,  0.24046607],
       [ 0.15127063,  0.98104077,  1.39618483,  0.24046607],
       [-1.89470363, -1.43517241, -1.157718  ,  0.24046607],
       ...,
       [ 0.56046548,  0.98104077,  1.39618483,  0.24046607],
       [ 0.69686376,  0.98104077,  1.39618483,  0.24046607],
       [ 0.42406719,  0.17563638,  0.11923341,  0.24046607]])

## Árboles de decisión (Decision Trees)

*Antes de pasar a la validación cruzada de K-Fold y los bosques aleatorios, comience creando una sola división de tren / prueba de nuestros datos. Reserve el 75% para la capacitación y el 25% para las pruebas.*

In [45]:
import numpy
from sklearn.model_selection import train_test_split

numpy.random.seed(1234)

(training_inputs,
 testing_inputs,
 training_classes,
 testing_classes) = train_test_split(valores_scaled,clases, train_size=0.75, random_state=1)

*Ahora cree un DecisionTreeClassifier y ajústelo a sus datos de entrenamiento.*

In [46]:
from sklearn.tree import DecisionTreeClassifier

clf= DecisionTreeClassifier(random_state=1)

# Train the classifier on the training set
clf.fit(training_inputs, training_classes)

DecisionTreeClassifier(random_state=1)

*Mida la precisión del modelo de árbol de decisión resultante utilizando los datos de prueba.*

In [47]:
clf.score(testing_inputs, testing_classes)

0.7355769230769231

*Ahora, en lugar de una sola división de train / test, use la validación cruzada K-Fold para obtener una mejor medida de la precisión de su modelo (K = 10). Sugerencia: use model_selection.cross_val_score*

In [48]:
from sklearn.model_selection import cross_val_score

clf = DecisionTreeClassifier(random_state=1)

cv_scores = cross_val_score(clf, valores_scaled, clases, cv=10)

cv_scores.mean()

0.7373493975903613

*Ahora pruebe un RandomForestClassifier en su lugar. ¿Funciona mejor?*

In [49]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=10, random_state=1)
cv_scores = cross_val_score(clf, valores_scaled, clases, cv=10)

cv_scores.mean()

0.7421686746987952

## SVM

*A continuación, intente usar svm. SVC con un núcleo lineal. ¿Cómo se compara con el árbol de decisión?*

In [50]:
from sklearn import svm

C = 1.0
svc = svm.SVC(kernel='linear', C=C)

cv_scores = cross_val_score(svc, valores_scaled, clases, cv=10)

cv_scores.mean()

0.7975903614457832

## KNN


*¿Qué hay de K-Nearest-Neighbors? Sugerencia: usa vecinos. KNeighborsClassifier - es mucho más fácil que implementar KNN desde cero como hicimos anteriormente en el curso. Comience con una K de 10. K es un ejemplo de un hiperparámetro, un parámetro en el propio modelo que puede necesitar ser ajustado para obtener los mejores resultados en su conjunto de datos particular.*

In [51]:
from sklearn import neighbors

import warnings
warnings.filterwarnings("ignore")

clf = neighbors.KNeighborsClassifier(n_neighbors=10)
cv_scores = cross_val_score(clf, valores_scaled, clases, cv=10)

cv_scores.mean()

0.7927710843373494

*Elegir K es complicado, por lo que no podemos descartar KNN hasta que hayamos probado diferentes valores de K. Escriba un bucle for para ejecutar KNN con valores K que van de 1 a 50 y vea si K hace una diferencia sustancial. Tome nota del mejor rendimiento que podría obtener de KNN.*

In [52]:
for n in range(1, 50):
    clf = neighbors.KNeighborsClassifier(n_neighbors=n)
    cv_scores = cross_val_score(clf, valores_scaled, clases, cv=10)
    print (n, cv_scores.mean())

1 0.7228915662650601
2 0.6855421686746987
3 0.7530120481927711
4 0.7385542168674699
5 0.7783132530120482
6 0.7650602409638554
7 0.7975903614457832
8 0.7819277108433734
9 0.7927710843373493
10 0.7927710843373494
11 0.7951807228915662
12 0.7843373493975905
13 0.7843373493975904
14 0.7855421686746988
15 0.7855421686746988
16 0.7831325301204819
17 0.7867469879518072
18 0.7783132530120482
19 0.7855421686746988
20 0.7843373493975904
21 0.7867469879518072
22 0.783132530120482
23 0.7795180722891566
24 0.7771084337349399
25 0.7855421686746988
26 0.7831325301204819
27 0.7843373493975904
28 0.7843373493975904
29 0.7867469879518072
30 0.7843373493975904
31 0.7867469879518072
32 0.789156626506024
33 0.7867469879518072
34 0.789156626506024
35 0.7843373493975904
36 0.7867469879518072
37 0.7831325301204819
38 0.7867469879518072
39 0.7819277108433734
40 0.7843373493975904
41 0.7819277108433734
42 0.7831325301204819
43 0.7831325301204819
44 0.7843373493975904
45 0.7831325301204819
46 0.7831325301204819


## Naive Bayes

Ahora prueba naive_bayes. MultinomialNB. ¿Cómo se compara su precisión?

In [53]:
from sklearn.naive_bayes import MultinomialNB

scaler = preprocessing.MinMaxScaler()
valores_minmax = scaler.fit_transform(valores)

clf = MultinomialNB()
cv_scores = cross_val_score(clf, valores_minmax, clases, cv=10)

cv_scores.mean()

0.7855421686746988

## Revisitando SVM

*SVM. SVC puede funcionar de manera diferente con diferentes kernels. La elección del núcleo es un ejemplo de un "hiperparámetro". Pruebe los kernels rbf, sigmoid y poly y vea cuál es el kernel de mejor rendimiento. ¿Tenemos un nuevo ganador?*

In [54]:
C = 1.0
svc = svm.SVC(kernel='rbf', C=C)
cv_scores = cross_val_score(svc, valores_scaled, clases, cv=10)
cv_scores.mean()

0.8012048192771084

In [55]:
C = 1.0
svc = svm.SVC(kernel='sigmoid', C=C)
cv_scores = cross_val_score(svc, valores_scaled, clases, cv=10)
cv_scores.mean()

0.7457831325301204

In [56]:
C = 1.0
svc = svm.SVC(kernel='poly', C=C)
cv_scores = cross_val_score(svc, valores_scaled, clases, cv=10)
cv_scores.mean()

0.7903614457831326

## Regresión logística

*Hemos probado todas estas técnicas sofisticadas, pero fundamentalmente esto es solo un problema de clasificación binaria. Pruebe la regresión logística, que es una forma sencilla de abordar este tipo de cosas.*

In [57]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
cv_scores = cross_val_score(clf, valores_scaled, clases, cv=10)
cv_scores.mean()

0.8072289156626505

## Redes neuronales

*Como desafío adicional, veamos si una red neuronal artificial puede hacerlo aún mejor. Puede usar Keras para configurar una red neuronal con 1 neurona de salida binaria y ver cómo funciona. No tenga miedo de ejecutar un gran número de épocas para entrenar el modelo si es necesario.*

In [58]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

def create_model():
    model = Sequential()
    #4 feature inputs going into an 6-unit layer (more does not seem to help - in fact you can go down to 4)
    model.add(Dense(6, input_dim=4, kernel_initializer='normal', activation='relu'))
    # "Deep learning" turns out to be unnecessary - this additional hidden layer doesn't help either.
    #model.add(Dense(4, kernel_initializer='normal', activation='relu'))
    # Output layer with a binary classification (benign or malignant)
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    # Compile model; adam seemed to work best
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [59]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# Wrap our Keras model in an estimator compatible with scikit_learn
estimator = KerasClassifier(build_fn=create_model, epochs=100, verbose=0)
# Now we can use scikit_learn's cross_val_score to evaluate this model identically to the others
cv_scores = cross_val_score(estimator, valores_scaled, clases, cv=10)
cv_scores.mean()

0.7987951815128327

**Podemos observar que en general todos los modelos propuestos nos entregan una precision de entre 79-80, excepto DecisionTree**